In [14]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [15]:
DIRTY = 1

In [16]:
class CleaningRobot(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        # Si la celda esta sucia, la limpia
        if self.model.grid[self.pos] == DIRTY:
            self.model.grid[self.pos] = 0
            self.model.cleaned += 1
        else:
            # Elige una nueva celda
            self.move()

    def move(self):
        # Da de alta sus posibles movimientos
        next_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Se mueve a dichas celdas
        next_move = self.random.choice(next_moves)
        # Intenta moverse a una nueva posicion
        if self.model.grid.is_cell_empty(next_move):
            self.model.grid.move_agent(self, next_move)

In [17]:
# def get_grid(model):

In [22]:
class CleaningModel(Model):
    def __init__(self, width, height, initial_dirt, n_agents):
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.cleaned = 0 # Es el contador de celdad limpias

        # Iniciamos celdad sucias
        for _, x, y in self.grid.coord_iter():
            if self.random.random() < initial_dirt:
                self.grid[x][y] = DIRTY

        for i in range(n_agents):
            robot = CleaningRobot(i, self)
            # Lo agentes empiezan en la celda 1, 1
            self.grid.place_agent(robot, (1,1))
            self.schedule.add(robot)

        self.datacollector = DataCollector(
            model_reporters={"Cleaned": lambda m: m.cleaned}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [23]:
width = 100
height = 100
initial_dirt = 0.9
n_agents = 1

model = CleaningModel(width, height, initial_dirt, n_agents)

while model.cleaned < width * height * (1 - initial_dirt):
    model.step()

TypeError: '<=' not supported between instances of 'int' and 'list'

In [ ]:
# all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
# fig, axs = plt.subplots(figsize=(7,7))
# axs.set_xticks([])
# axs.set_yticks([])
# patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

# def animate(i):
#     patch.set_data(all_grid.iloc[i][0])
    
# anim = animation.FuncAnimation(fig, animate, frames=MAX_GENERATIONS)

In [ ]:
# anim